In [23]:
# Execute this prior to running test
import nltk
nltk.download('punkt')
nltk.download('stopwords')
 

[nltk_data] Downloading package punkt to /Users/drew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/drew/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [73]:
# for the four data sets 
# loop 0 to 35 and if in the sentence, replace it with the word's value from wordtovec
# if no word then put in 0 instead
TRAINING_OFFENSIVE_FILENAME = "./training_offensive.csv"
TRAINING_REGULAR_FILENAME = "./training_regular.csv"
TEST_OFFENSIVE_FILENAME = "./test_offensive.csv"
TEST_REGULAR_FILENAME = "./test_regular.csv"
LOCAL_TRAINING_OFFENSIVE_FILENAME = "./training_offensive_local.csv"
LOCAL_TRAINING_REGULAR_FILENAME = "./training_regular_local.csv"
LOCAL_TEST_OFFENSIVE_FILENAME = "./test_offensive_local.csv"
LOCAL_TEST_REGULAR_FILENAME = "./test_regular_local.csv"

#FB_MODEL_FILENAME = "./cc.en.300.bin"
#with open(LOCAL_TEST_REGULAR_FILENAME) as f:
#    print(f.read())
# fb_model = load_facebook_model(FB_MODEL_FILENAME) # do this in real code
# off_train_dataframe = pandas.read_csv(TRAINING_OFFENSIVE_FILENAME)
# off_test_dataframe = pandas.read_csv(TEST_OFFENSIVE_FILENAME)
# reg_train_dataframe = pandas.read_csv(TRAINING_REGULAR_FILENAME)
# reg_test_dataframe = pandas.read_csv(TEST_REGULAR_FILENAME)
off_train_dataframe = pandas.read_csv(LOCAL_TRAINING_OFFENSIVE_FILENAME)
off_test_dataframe = pandas.read_csv(LOCAL_TEST_OFFENSIVE_FILENAME)
reg_train_dataframe = pandas.read_csv(LOCAL_TRAINING_REGULAR_FILENAME)
reg_test_dataframe = pandas.read_csv(LOCAL_TEST_REGULAR_FILENAME)

In [74]:
stop_words = set(stopwords.words('english')) 
non_alphabet_pattern = re.compile("[^a-zA-Z]")
#NULL_VECTOR = fb_model.wv[""]

def get_cleaned_tokens(sentence):    
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [re.sub(non_alphabet_pattern, "", w) for w in filtered_sentence]
    filtered_sentence = [w for w in filtered_sentence if w != ""]
    return filtered_sentence


In [93]:
# Create Training Tweets
offensive_train_tweets = off_train_dataframe['tweet']
non_offensive_train_tweets = reg_train_dataframe['tweet']
# offensive_train_tweets = [get_cleaned_tokens(tweet) for tweet in off_train_dataframe['tweet']]
# non_offensive_train_tweets = [get_cleaned_tokens(tweet) for tweet in reg_train_dataframe['tweet']]
all_train_tweets = offensive_train_tweets + non_offensive_train_tweets

# Create Traianing Labels
offensive_train_labels = off_train_dataframe['offensive']
non_offensive_train_labels = reg_train_dataframe['offensive']
all_train_labels = offensive_train_labels + non_offensive_train_labels


# Create Test Tweets
offensive_test_tweets = off_test_dataframe['tweet']
non_offensive_test_tweets = reg_test_dataframe['tweet']
# offensive_test_tweets = [get_cleaned_tokens(tweet) for tweet in off_test_dataframe['tweet']]
# non_offensive_test_tweets = [get_cleaned_tokens(tweet) for tweet in reg_test_dataframe['tweet']]
all_test_tweets = offensive_test_tweets + non_offensive_test_tweets

# Create Test Labels
offensive_test_labels = off_test_dataframe['offensive']
non_offensive_test_labels = reg_test_dataframe['offensive']
all_test_labels = offensive_test_labels + non_offensive_test_labels

print(all_train_tweets)

0     !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
1     !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
2     !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
3     !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
4     !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...
5     !!!!!!"@__BrighterDays: I can not just sit up ...
6     !!!!&#8220;@selfiequeenbri: cause I'm tired of...
7     " &amp; you might not get ya bitch back &amp; ...
8     " @rhythmixx_ :hobbies include: fighting Maria...
9     " Keeks is a bitch she curves everyone " lol I...
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                                  NaN
15                                                  NaN
16                                                  NaN
17                                              

In [92]:
# tokenize words 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
VOCAB_SIZE = 100000
tokenizer = Tokenizer(nb_words=VOCAB_SIZE)
tokenizer.fit_on_texts(all_train_tweets)

sequences_train = tokenizer.texts_to_sequences(all_train_tweets)
sequences_test = tokenizer.texts_to_sequences(all_test_tweets)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


AttributeError: 'float' object has no attribute 'lower'

In [77]:
# pad sequences
train_data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(sequences_test, maxlen = MAX_SEQUENCE_LENGTH)

In [80]:
# print(len(all_train_tweets))
# print(len(all_test_tweets))
# print(len(all_train_labels))
# print(len(all_test_labels))

In [81]:
embeddings_index = {}
import os
FILEPATH = "./glove.twitter.27B.50d.txt"
f = open(os.path.join(FILEPATH))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [82]:
from numpy import array
from numpy import asarray
from numpy import zeros
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((VOCAB_SIZE, 50)) #DIMENSIONS!
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    

Loaded 1193514 word vectors.


In [84]:
"""
MAIN MODEL
"""
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalMaxPooling1D
# define model
model = Sequential()
e = Embedding(VOCAB_SIZE, 50, weights=[embedding_matrix], input_length=70, trainable=False)
model.add(e)
model.add(Flatten()) # must use Flatter or GlobalMax
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(train_data, all_train_labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(test_data, all_test_labels, verbose=0)
print("Loss: {}".format(loss))
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 70, 50)            5000000   
_________________________________________________________________
flatten_16 (Flatten)         (None, 3500)              0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                35010     
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 5,035,021
Trainable params: 35,021
Non-trainable params: 5,000,000
_________________________________________________________________
None
Loss: 0.3887756029764811
Accuracy: 83.333331


In [ ]:
# model = keras.Sequential()
# model.add(keras.layers.Conv1D(filter_num, kernel_size, activation = 'relu'))
# model.add(keras.layers.GlobalMaxPooling1D())
# model.add(keras.layers.Dense(10, activation='relu'))
# model.add(keras.layers.Dense(1, activation='sigmoid'))
# model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# history = model.fit(offensive_vectors, non_offensive_vectors,
#                     epochs=10,
#                     verbose=False,
#                     validation_data=(offensive_vectors_test, non_offensive_vectors_test),
#                     batch_size=10)
# loss, accuracy = model.evaluate(offensive_vectors, non_offensive_vectors, verbose=False)
# print("Training Accuracy: {:.4f}".format(accuracy))
# loss, accuracy = model.evaluate(offensive_vectors_test, non_offensive_vectors_test, verbose=False)
# print("Testing Accuracy:  {:.4f}".format(accuracy))
# plot_history(history)

In [85]:
# from numpy import array
# from numpy import asarray
# from numpy import zeros
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers import Embedding
# # define documents
# docs = ['Well done!',
# 		'Good work',
# 		'Great effort',
# 		'nice work',
# 		'Excellent!',
# 		'Weak',
# 		'Poor effort!',
# 		'not good',
# 		'poor work',
# 		'Could have done better.']
# # define class labels
# labels = array([1,1,1,1,1,0,0,0,0,0])
# # prepare tokenizer
# t = Tokenizer()
# t.fit_on_texts(docs)
# vocab_size = len(t.word_index) + 1
# # integer encode the documents
# encoded_docs = t.texts_to_sequences(docs)
# print(encoded_docs)
# # pad documents to a max length of 4 words
# max_length = 4
# padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(padded_docs)
# # load the whole embedding into memory
# embeddings_index = dict()
# f = open('../glove_data/glove.6B/glove.6B.100d.txt')
# for line in f:
# 	values = line.split()
# 	word = values[0]
# 	coefs = asarray(values[1:], dtype='float32')
# 	embeddings_index[word] = coefs
# f.close()